# Lab 03: Fun with `pandas`!

Below are some exercises to get you working with `pandas` to manipulate data. As always, get as far as you can, and ask for help when you need it! Your teacher (me), you instructor, and your classmates are all here to help each other get better at coding. Getting the code to work is important, but do also take the time to make sure you understand what the commands are doing. This time, (with the exception of the Stroop challenge), all I've given you is the code to download the data. Then you are on your own. For the Stroop challenge, I gave the you code for the first step—after that, it's up to you :-)

## Music sales challenge

Write a script that:

1. Combines the tables of best-selling physical singles and best-selling digital singles on the Wikipedia page "List_of_best-selling_singles"
2. Adds a column which marks whether each row is from the list of physical singles or digital singles
3. Outputs the artist and single name for the year you were born. If there is no entry for that year, take the closest year after you were born.
4. Outputs the artist and single name for the year you were 15 years old.

In [46]:
# Starter code...

#%pip install lxml
import pandas as pd

rawdata = pd.read_html("https://en.wikipedia.org/wiki/List_of_best-selling_singles")

# after inspection, rawdata seem to be a list of dataframes
rawdata[1].head()

# strucutre of the physical and digital sales data are identical

# finding the dataframes which only has these columns Artist, Single, Released, Sales(in millions), Source
for i in range(len(rawdata)):
    if set(['Artist', 'Single', 'Released', 'Sales (in millions)', 'Source']).issubset(rawdata[i].columns):
        print(f"The following dataframe (index {i}) has the required columns:")

# making dataframes for physical and digital sales
physical_singles_1 = rawdata[0]
physical_singles_2 = rawdata[1]
digital_singles_1 = rawdata[3]
digital_singles_2 = rawdata[4]

# combining the dataframes separately
physical_singles = pd.concat([physical_singles_1, physical_singles_2], ignore_index=True)
digital_singles = pd.concat([digital_singles_1, digital_singles_2], ignore_index=True)

# adding a column 'Type' to the dataframes to distinguish between physical and digital sales
physical_singles['Type'] = 'Physical'
digital_singles['Type'] = 'Digital'

# combinging the two dataframes with concat
combined_singles = pd.concat([physical_singles, digital_singles], ignore_index=True)

# finding all songs from 1998-2000 (my birth year is 1999). First Released should be a string
combined_singles['Released'] = combined_singles['Released'].astype(str)
# finding all songs from 1998-2000
songs_1998_2000 = combined_singles[combined_singles['Released'].str.contains('1998|1999|2000')]
# Printing artist and single name and year of release
print(songs_1998_2000[['Artist', 'Single', 'Released']])

# finding the best selling singles when i was 15 in 2014
songs_2014 = combined_singles[combined_singles['Released'].str.contains('2014')]
# sorting the songs by sales
songs_2014 = songs_2014.sort_values(by='Sales (in millions)', ascending=False)
# Printing artist and single name and year of release
print(songs_2014[['Artist', 'Single', 'Released']])



The following dataframe (index 0) has the required columns:
The following dataframe (index 1) has the required columns:
The following dataframe (index 3) has the required columns:
The following dataframe (index 4) has the required columns:
            Artist                   Single Released
25            Cher                "Believe"     1998
42  Britney Spears  "...Baby One More Time"     1998
            Artist                 Single Released
91  Meghan Trainor  "All About That Bass"     2014


### Best selling song when i was born in 1999 was:
Cher, "Believe" 1998 and Britney Spears, "...Baby One More Time" 1998

### Best selling song when i was 15 years old in 2014 was:
Meghan Trainor, "All About That Bass" 2014

## Space challenge

1. Make a single dataframe that combines the space missions from the 1950's to the 2020's
2. Write a script that returns the year with the most launches
3. Write a script that returns the most common month for launches
4. Write a script that ranks the months from most launches to fewest launches


In [72]:
# Starter code...

#%pip install lxml
import pandas as pd

rawdata = pd.read_html("https://en.wikipedia.org/wiki/Timeline_of_Solar_System_exploration")

# after inspection, rawdata seem to be a list of dataframes whereas 0-7 are the tables we are interested in

# Making a list of dataframes for the tables we are interested in
space_missions = rawdata[0:8]
# concatting to make it a pd dataframe
space_missions = pd.concat(space_missions, ignore_index=True)
space_missions.head()

# finding year with most missions (launch date is in format 4 october 1957)
space_missions['Launch date'] = pd.to_datetime(space_missions['Launch date'], errors='coerce')
# finding the year with most missions
year_with_most_missions = space_missions['Launch date'].dt.year.value_counts().idxmax()
print(f"The year with most missions is {year_with_most_missions}")

# finding month with most common launches
most_common_month = space_missions['Launch date'].dt.month.value_counts().idxmax()

# converting month name so that it is not a nu,ber
month_name = pd.to_datetime(most_common_month, format='%m').strftime('%B')
print(f"Most common month for launch is: {month_name}")

# Finding the number of launches per month, sorted from fewest to most
launches_per_month = space_missions['Launch date'].dt.month.value_counts().sort_values(ascending=True)

# Mapping month numbers to month names for better readability
launches_per_month.index = launches_per_month.index.map(lambda x: pd.to_datetime(f"2024-{x}-01", format='%Y-%m-%d').strftime('%B'))

print("Launches per month (fewest to most):")
print(launches_per_month)


The year with most missions is 1965
Most common month for launch is: November
Launches per month (fewest to most):
Launch date
April        13
February     14
June         14
March        15
May          17
January      19
December     19
July         21
October      22
September    22
August       27
November     30
Name: count, dtype: int64


### Year with most missions:
1965

### Most common month of launches:
November

### List of launches per month (ascending):
April:        13,
February:     14,
June:         14,
March:        15,
May:          17,
January:      19,
December:     19,
July:         21,
October:      22,
September:    22,
August:       27,
November:     30

## Supervillain challenge

1. Write a script that combines the tables showing supervillain debuts from the 30's through the 2010's
2. Write a script that ranks each decade in terms of how many supervillains debuted in that decade
3. Write a script that ranks the different comics companies in terms of how many supervillains they have, and display the results in a nice table (pandas dataframe)

In [95]:
supervillains = pd.read_html("https://en.wikipedia.org/wiki/List_of_comic_book_supervillain_debuts")

# From the looks of the data, we concat table 3-11
villains = pd.concat(supervillains[3:12], ignore_index=True)

# Ranking decades by number of supervillain debuts. First removing everything else but 3 first character of year and adding a 0
villains['Year Debuted'] = villains['Year Debuted'].astype(str)
villains['Decade'] = villains['Year Debuted'].str[:3] + '0'
decade_ranking = villains['Decade'].value_counts()
# removing all rows in decade not starting with 3 numbers (using regex letters)
decade_ranking = decade_ranking[decade_ranking.index.str.match(r'\d{3}')]
# Printing the decades by number of supervillain debuts
print("Ranking of decades by number of supervillain debuts:")
print(decade_ranking)

# making a dataframe including the company and number of supervillains
company_ranking = (villains['Company'].value_counts().reset_index(name='Number of Supervillains').rename(columns={'index': 'Company'}))

# displaying the dataframe
company_ranking  

Ranking of decades by number of supervillain debuts:
Decade
1960    228
1970     96
1980     90
1990     83
2000     49
1940     46
1950     26
2010      9
1930      4
Name: count, dtype: int64


,Company,Number of Supervillains
0,DC,338
1,Marvel,264
2,Fawcett Comics/DC,6
3,Image,5
4,Dark Horse,5
5,Marvel/Timely,4
6,Disney/Hyperion,4
7,Eternity,3
8,Lev Gleason Publications,1
9,Comico,1


## Stroop challenge

Every year between 2015 and 2021, the students in my Language, Cognition, and the Brain course participated in a version of the Stroop task. Using a stopwatch (ok, using their phones), they recorded how fast they could say a list of things (either reading or naming colors or color words). The column names mean "Reading with No Interference", "Naming with Interference", "Naming with No Interference", and "Reading with Interference". The times are in seconds.

### Stroop challenge 1: 
Transform these data from wide format to long format, so that the result is a dataframe with
- 1 column named "Participant_id" with a unique number for each participant (you can use the row indices)
- 1 column named "Year" with the year data
- 1 column named "Task" that shows which task they were doing
- 1 column named "RT" that shows their response time

In [104]:
import pandas as pd

# Reading the data from the provided URL
df = pd.read_csv("https://raw.githubusercontent.com/ethanweed/Stroop/master/Stroop-raw-over-the-years.csv")

# adding a Participant_id column based on the row index
df['Participant_id'] = df.index + 1

# using pandas melt to transform dataframe to long format. Keeping participant_id and year the same. 
df_long = pd.melt(df, 
                  id_vars=['Participant_id', 'Year'],  # keeping these columns as they are
                  value_vars=['Reading_NoInt', 'Naming_Int', 'Naming_NoInt', 'Reading_Int'],  # columns to melt
                  var_name='Task',  # new column for task
                  value_name='RT')  # new column for reaction times

# display the transformed dataframe
df_long.head()


,Participant_id,Year,Task,RT
0,1,2015,Reading_NoInt,4.16
1,2,2015,Reading_NoInt,4.35
2,3,2015,Reading_NoInt,3.60
3,4,2015,Reading_NoInt,3.90
4,5,2015,Reading_NoInt,4.22


## Stroop challenge 2 (Advanced!!!):

Make a new dataframe which shows the mean response time (in seconds) for each task for each year.

In [110]:
# First we have to calculate mean response time for each task and year. We do this by grouping by year and task and then calculating the mean.
# After calcuating the mean, we unstack the task column (and making it wide-format again).
mean_rt = df_long.groupby(['Year', 'Task'])['RT'].mean().unstack()
#resetting the index
mean_rt = mean_rt.reset_index()
# removing the index name
mean_rt.columns.name = None
mean_rt

,Year,Naming_Int,Naming_NoInt,Reading_Int,Reading_NoInt
0,2015,8.617143,5.123571,4.446429,3.951429
1,2016,8.859268,5.405610,5.340000,4.076098
2,2017,9.311765,5.771176,5.492353,4.414412
3,2018,9.372667,5.298000,4.938667,3.886000
4,2019,9.536087,6.345652,6.090870,4.935652
5,2020,9.740833,5.962500,4.956667,4.395000
6,2021,10.105484,6.387742,7.038065,4.842581
